#### Script to check rf and compute the visuomemory index block 1
- First, check if neuron responding to the task by comparing activity during trial vs baseline
- Second, check whether the rf is in in or out
- Third, compute the visuomemory index

In [27]:
import numpy as np
from matplotlib import pyplot as plt
from ephysvibe.trials.spikes import firing_rate
from ephysvibe.trials import align_trials
from ephysvibe.task import task_constants
from ephysvibe.structures.neuron_data import NeuronData
from scipy import stats

### Def functions

### Start analysis

In [28]:
neu_path = "/home/INT/losada.c/Documents/session_struct/v4/neurons/2023-03-03_10-59-32_Riesling_v4_e1_r1_mua7_neu.h5"
neu_data = NeuronData.from_python_hdf5(neu_path)

#### Select parameters

In [29]:
# parameters
time_before=500
select_block=2
start=-200
end=1200
idx_start = time_before + start 
idx_end = time_before+end
# ---conv
fs = 1000
w_size=0.1
w_std=0.015
# Timings 
## fixation
dur_fix = 200
## visual stim
st_v =   80
end_v = 300
## delay
st_d = 600 
end_d = 800
# trials and threshold
min_trials = 3
n_spikes = 1
p_threshold = 0.05
vd_threshold = 0.4

In [30]:
kernel = firing_rate.define_kernel(
        w_size, w_std, fs=fs
    )


In [31]:
# select correct trials, block one, inside RF, and align with sample onset 
sp_sample_on_in,mask_in = align_trials.align_on(
    sp_samples=neu_data.sp_samples,
    code_samples=neu_data.code_samples,
    code_numbers=neu_data.code_numbers,
    trial_error=neu_data.trial_error,
    block=neu_data.block,
    pos_code=neu_data.pos_code,
    select_block= 1,
    select_pos= 1,
    event ="sample_on",
    time_before = time_before,
    correct_trials= True,
)
conv_in = np.apply_along_axis(lambda m: np.convolve(m, kernel, mode='same'), axis=1, arr=sp_sample_on_in)* fs
conv_in = conv_in[:,idx_start:idx_end]

In [32]:
# select correct trials, block one, outside RF, and align with sample onset 
sp_sample_on_out,mask_out = align_trials.align_on(
    sp_samples=neu_data.sp_samples,
    code_samples=neu_data.code_samples,
    code_numbers=neu_data.code_numbers,
    trial_error=neu_data.trial_error,
    block=neu_data.block,
    pos_code=neu_data.pos_code,
    select_block= 1,
    select_pos= -1,
    event ="sample_on",
    time_before = time_before,
    correct_trials= True,
)
conv_out = np.apply_along_axis(lambda m: np.convolve(m, kernel, mode='same'), axis=1, arr=sp_sample_on_out)* fs
conv_out=conv_out[:,idx_start:idx_end]

### select neuron RF:
* 1: compare whether activity during sample or delay (in condition) is significant and larger compared to the baseline
* 2: if significant -> in, if not -> check if out condition is significant
* 3: if significant -> out, if not -> in

In [33]:

def check_activity(conv,dur_fix,st_v,end_v,st_d,end_d):
    if conv.shape[0] > min_trials:
        mean_visual = conv[:, dur_fix + st_v : dur_fix + end_v].mean(axis=1)
        mean_delay = conv[:, dur_fix + st_d : dur_fix + end_d].mean(axis=1)
        mean_bl = conv[:, :dur_fix].mean(axis=1)

        v_larger = mean_bl.mean() < mean_visual.mean()
        d_larger = mean_bl.mean() < mean_delay.mean()

        # paired sample t-test: compare baseline vs epochs
        p_v = stats.ttest_rel(mean_bl, mean_visual)[1]
        p_d = stats.ttest_rel(mean_bl, mean_delay)[1]
    else:
        v_larger,d_larger = False,False
        p_v,p_d = np.nan, np.nan
    return v_larger,d_larger,p_v,p_d


In [34]:
v_larger_in,d_larger_in,p_v_in,p_d_in = check_activity(conv_in,dur_fix,st_v,end_v,st_d,end_d)
v_larger_out,d_larger_out,p_v_out,p_d_out = check_activity(conv_out,dur_fix,st_v,end_v,st_d,end_d)

In [35]:
check_in = np.logical_and(np.logical_or(v_larger_in,d_larger_in),np.logical_or(p_v_in<p_threshold,p_d_in<p_threshold))
check_out = np.logical_and(np.logical_or(v_larger_out,d_larger_out),np.logical_or(p_v_out<p_threshold,p_d_out<p_threshold))
if check_in:
    rf = 'in'
    significant = True
elif check_out:
    rf='out'
    significant = True
else:
    rf='in'
    significant = False
    

#### Compute VD index

In [36]:
if rf == 'in':
    conv = conv_in
elif rf == 'out':
    conv = conv_out
if significant:
    mean_visual = conv[:, dur_fix + st_v : dur_fix + end_v].mean()
    mean_delay = conv[:, dur_fix + st_d : dur_fix + end_d].mean()
    max_fr = conv[:,  : dur_fix + end_d].mean()
    mean_delay = mean_delay / max_fr  
    mean_visual = mean_visual / max_fr  
    vd_index = (mean_delay - mean_visual) / (mean_delay + mean_visual)
else:
    vd_index=np.nan

In [39]:
# neu_data.vd_index=vd_index
# neu_data.to_python_hdf5('ddd.h5')
# dd = NeuronData.from_python_hdf5('ddd.h5')